# 0- Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
try:
    from amplpy import AMPL, tools
except ImportError:
    # Caso ocorra um erro ao importar, instala amplpy e tenta importar novamente
    %pip install amplpy
    from amplpy import AMPL, tools

In [3]:
licenca ="default"

In [4]:

ampl = tools.ampl_notebook(
    modules=["highs", "gokestrel", "gurobi", "cplex"],  # Módulos a serem instalados
    license_uuid=licenca,  # Licença a ser utilizada
    g=globals()  # Instancia o objeto AMPL e registra os comandos mágicos
)


# 1- Base

Importando a base pré-processada com o limite de produção diário de:
<ul>
    <li>120.000 peças para o “Tipo de buffer” CX e </li>
    <li>100.000 peças para o “Tipo de buffer” EN.</li>
 </ul>

In [5]:
df_process = pd.read_excel('../../01-Data/ordens_pre_process_revisado.xlsx')

df_process

,Unnamed: 0,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Tipo_de_buffer,Item_pai,Cor,Tamanho,Inner,Rota,Capacidade,pai_cor,chave_loja_buffer_onda,Volume_unit
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,CX,1865,6.0,27,1,1,inf,1865_6.0,26_CX_5,3092300
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,CX,1865,6.0,21,1,1,inf,1865_6.0,26_CX_5,3321500
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,CX,1865,6.0,4,1,1,inf,1865_6.0,26_CX_5,3537200
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,CX,2489,674.0,179,1,3,inf,2489_674.0,116_CX_6,233624
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,CX,2988,451.0,109,1,1,inf,2988_451.0,16_CX_2,775500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,EN,1863,2.0,38,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,EN,1863,2.0,58,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,EN,1863,2.0,16,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,EN,713,275.0,16,1,15,4.000000e+10,713_275.0,48_EN_2,4235707


In [6]:
df_process = df_process.rename(columns={'Unnamed: 0': "Id"})

In [7]:
df_process = df_process.rename(columns={'Capacidade': "Capacidade_rota"})

In [8]:
df_process.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade_rota', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit'],
      dtype='object')

In [9]:
df_process['tempo_espera_dias'] = (df_process['Data_da_ordem_de_produção'].max() - df_process['Data_da_ordem_de_produção']).dt.days.astype('int16')

In [10]:
# Definir a função para calcular a prioridade
def calcular_prioridade(delta_t):
    diferenca = delta_t_maximo - delta_t
    if diferenca <= 5:
        return delta_t_maximo
    elif diferenca <= 10:
        return delta_t_maximo - 10
    else:
        return delta_t_maximo - 20

In [11]:
delta_t_maximo = df_process['tempo_espera_dias'].max()
df_process['prioridade'] = df_process['tempo_espera_dias'].apply(calcular_prioridade)
df_process

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Cor,Tamanho,Inner,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,6.0,27,1,1,inf,1865_6.0,26_CX_5,3092300,98,98
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,6.0,21,1,1,inf,1865_6.0,26_CX_5,3321500,98,98
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,6.0,4,1,1,inf,1865_6.0,26_CX_5,3537200,98,98
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,674.0,179,1,3,inf,2489_674.0,116_CX_6,233624,37,78
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,451.0,109,1,1,inf,2988_451.0,16_CX_2,775500,37,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,2.0,38,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280,0,78
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,2.0,58,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285,0,78
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,2.0,16,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360,0,78
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,275.0,16,1,15,4.000000e+10,713_275.0,48_EN_2,4235707,0,78


In [12]:
(df_process['Data_da_ordem_de_produção'].max() - df_process['Data_da_ordem_de_produção']).dt.days.astype('int16').head(300)

0      98
1      98
2      98
3      37
4      37
       ..
295    20
296    20
297    20
298    20
299    20
Name: Data_da_ordem_de_produção, Length: 300, dtype: int16

In [13]:
df_process.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62231 entries, 0 to 62230
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         62231 non-null  int64         
 1   Ordem_de_produção          62231 non-null  int64         
 2   Data_da_ordem_de_produção  62231 non-null  datetime64[ns]
 3   Produto                    62231 non-null  int64         
 4   Comprimento_unit           62231 non-null  int64         
 5   Largura_unit               62231 non-null  int64         
 6   Altura_unit                62231 non-null  int64         
 7   Peso_unit                  62231 non-null  float64       
 8   Peças                      62231 non-null  int64         
 9   Caixa_padrão               62231 non-null  object        
 10  Loja                       62231 non-null  int64         
 11  Classe_de_onda             62231 non-null  int64         
 12  Tipo

In [14]:
df_process

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Cor,Tamanho,Inner,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,6.0,27,1,1,inf,1865_6.0,26_CX_5,3092300,98,98
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,6.0,21,1,1,inf,1865_6.0,26_CX_5,3321500,98,98
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,6.0,4,1,1,inf,1865_6.0,26_CX_5,3537200,98,98
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,674.0,179,1,3,inf,2489_674.0,116_CX_6,233624,37,78
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,451.0,109,1,1,inf,2988_451.0,16_CX_2,775500,37,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,2.0,38,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280,0,78
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,2.0,58,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285,0,78
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,2.0,16,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360,0,78
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,275.0,16,1,15,4.000000e+10,713_275.0,48_EN_2,4235707,0,78


## Caixa padrão

In [15]:
df_caixas_tipo = pd.read_excel('../../01-Data/Dados - desafio disciplina RPVMM_edit.xlsx', sheet_name='Tipos de caixa')
df_caixas_tipo

,Tipo,Tipo de buffer,Comprimento,Largura,Altura,Fator de ocupação,Peças max,Peso max (kg)
0,Caixa 1,CX,380,285,285,0.9,170,23
1,Caixa 2,CX,586,436,272,1.2,170,23
2,Caixa 3,CX,728,427,270,0.9,170,23
3,Caixa 1,EN,380,285,285,0.9,512,23
4,Caixa 2,EN,586,436,272,1.2,512,23
5,Caixa 3,EN,728,427,270,0.9,512,23


In [16]:
df_caixas_tipo.columns = df_caixas_tipo.columns.str.replace(' ', '_')

In [17]:
df_caixas_tipo['volume_util_caixa'] = df_caixas_tipo['Comprimento'] * df_caixas_tipo['Largura'] * df_caixas_tipo['Altura'] * df_caixas_tipo['Fator_de_ocupação']

df_caixas_tipo

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
0,Caixa 1,CX,380,285,285,0.9,170,23,27778950.0
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
2,Caixa 3,CX,728,427,270,0.9,170,23,75538008.0
3,Caixa 1,EN,380,285,285,0.9,512,23,27778950.0
4,Caixa 2,EN,586,436,272,1.2,512,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


In [18]:
df_caixas_tipo.pivot(index='Tipo_de_buffer', columns=['Tipo','volume_util_caixa'], values=['Peças_max', 'volume_util_caixa']).reset_index()

Tipo_de_buffer  Peças_max                        \
Tipo                                Caixa 1    Caixa 2    Caixa 3   
volume_util_caixa                27778950.0 83393894.4 75538008.0   
0                             CX      170.0      170.0      170.0   
1                             EN      512.0      512.0      512.0   

                  volume_util_caixa                          
Tipo                        Caixa 1     Caixa 2     Caixa 3  
volume_util_caixa        27778950.0  83393894.4  75538008.0  
0                        27778950.0  83393894.4  75538008.0  
1                        27778950.0  83393894.4  75538008.0

In [19]:
df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='volume_util_caixa').add_prefix('volume_util_caixa_').reset_index()

Tipo,Tipo_de_buffer,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,CX,27778950.0,83393894.4,75538008.0
1,EN,27778950.0,83393894.4,75538008.0


In [20]:
df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='Peças_max').add_prefix('Peças_max_').reset_index()

Tipo,Tipo_de_buffer,Peças_max_Caixa 1,Peças_max_Caixa 2,Peças_max_Caixa 3
0,CX,170,170,170
1,EN,512,512,512


In [21]:
df_caixas_tipo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Tipo               6 non-null      object 
 1   Tipo_de_buffer     6 non-null      object 
 2   Comprimento        6 non-null      int64  
 3   Largura            6 non-null      int64  
 4   Altura             6 non-null      int64  
 5   Fator_de_ocupação  6 non-null      float64
 6   Peças_max          6 non-null      int64  
 7   Peso_max_(kg)      6 non-null      int64  
 8   volume_util_caixa  6 non-null      float64
dtypes: float64(2), int64(5), object(2)
memory usage: 564.0+ bytes


In [22]:
df_caixas_tipo.columns

Index(['Tipo', 'Tipo_de_buffer', 'Comprimento', 'Largura', 'Altura',
       'Fator_de_ocupação', 'Peças_max', 'Peso_max_(kg)', 'volume_util_caixa'],
      dtype='object')

Temos um tipo de caixa padrão para cada tipo de buffer

In [23]:
df_caixas_padrao = df_caixas_tipo.loc[[1, 5]]
df_caixas_padrao

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


Levar a capacidade das caixas para cada ordem de produção:

O **número de peças máximo** varia com o buffer. Já o *volume* é igual para ambos os tipos, mudando apenas para o **tipo de caixa**

In [24]:
df_caixa_volume_max= df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='volume_util_caixa').add_prefix('volume_util_caixa_').reset_index()

df_caixa_volume_max

Tipo,Tipo_de_buffer,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,CX,27778950.0,83393894.4,75538008.0
1,EN,27778950.0,83393894.4,75538008.0


In [25]:
df_process_capacitado = df_process.merge(df_caixa_volume_max, on='Tipo_de_buffer', how='inner')

df_process_capacitado

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,1,inf,1865_6.0,26_CX_5,3092300,98,98,27778950.0,83393894.4,75538008.0
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,1,inf,1865_6.0,26_CX_5,3321500,98,98,27778950.0,83393894.4,75538008.0
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,1,inf,1865_6.0,26_CX_5,3537200,98,98,27778950.0,83393894.4,75538008.0
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,3,inf,2489_674.0,116_CX_6,233624,37,78,27778950.0,83393894.4,75538008.0
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,1,inf,2988_451.0,16_CX_2,775500,37,78,27778950.0,83393894.4,75538008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,15,4.000000e+10,1863_2.0,48_EN_2,3269280,0,78,27778950.0,83393894.4,75538008.0
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,15,4.000000e+10,1863_2.0,48_EN_2,4037285,0,78,27778950.0,83393894.4,75538008.0
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,15,4.000000e+10,1863_2.0,48_EN_2,3231360,0,78,27778950.0,83393894.4,75538008.0
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,15,4.000000e+10,713_275.0,48_EN_2,4235707,0,78,27778950.0,83393894.4,75538008.0


In [26]:
df_process_capacitado.columns = df_process_capacitado.columns.str.replace(' ', '_')

In [27]:
import numpy as np
df_process_capacitado['volume_util_padrao'] = np.where(df_process_capacitado['Tipo_de_buffer'] == "EN", df_process_capacitado['volume_util_caixa_Caixa_3'], df_process_capacitado['volume_util_caixa_Caixa_2'])

In [28]:
df_process_capacitado = df_process_capacitado.merge(df_caixas_padrao[['Tipo_de_buffer', 'Peças_max']], on='Tipo_de_buffer', how='inner')
df_process_capacitado

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,1865_6.0,26_CX_5,3092300,98,98,27778950.0,83393894.4,75538008.0,83393894.4,170
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,1865_6.0,26_CX_5,3321500,98,98,27778950.0,83393894.4,75538008.0,83393894.4,170
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,1865_6.0,26_CX_5,3537200,98,98,27778950.0,83393894.4,75538008.0,83393894.4,170
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,2489_674.0,116_CX_6,233624,37,78,27778950.0,83393894.4,75538008.0,83393894.4,170
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,2988_451.0,16_CX_2,775500,37,78,27778950.0,83393894.4,75538008.0,83393894.4,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,1863_2.0,48_EN_2,3269280,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,1863_2.0,48_EN_2,4037285,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,1863_2.0,48_EN_2,3231360,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,713_275.0,48_EN_2,4235707,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512


In [29]:
df_process_capacitado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62231 entries, 0 to 62230
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         62231 non-null  int64         
 1   Ordem_de_produção          62231 non-null  int64         
 2   Data_da_ordem_de_produção  62231 non-null  datetime64[ns]
 3   Produto                    62231 non-null  int64         
 4   Comprimento_unit           62231 non-null  int64         
 5   Largura_unit               62231 non-null  int64         
 6   Altura_unit                62231 non-null  int64         
 7   Peso_unit                  62231 non-null  float64       
 8   Peças                      62231 non-null  int64         
 9   Caixa_padrão               62231 non-null  object        
 10  Loja                       62231 non-null  int64         
 11  Classe_de_onda             62231 non-null  int64         
 12  Tipo

In [30]:
df_process_capacitado['chave_loja_buffer_onda'].nunique()

1508

In [31]:
df_process_capacitado.query('Tipo_de_buffer == "EN"').groupby('chave_loja_buffer_onda')['Peças'].sum().sort_values(ascending=False).head(20)

chave_loja_buffer_onda
10_EN_3    3459
28_EN_4    3187
16_EN_3    2093
2_EN_3     2086
31_EN_3    1802
33_EN_3    1786
8_EN_4     1620
10_EN_4    1509
9_EN_4     1464
3_EN_3     1403
25_EN_4    1367
18_EN_4    1360
48_EN_4    1355
8_EN_3     1348
10_EN_1    1329
23_EN_3    1299
11_EN_4    1228
3_EN_4     1162
2_EN_4     1156
9_EN_3     1127
Name: Peças, dtype: int64

In [32]:
(df_process_capacitado.query('Tipo_de_buffer == "EN"').groupby('chave_loja_buffer_onda')['Peças'].sum().sort_values(ascending=False).head(20)).index

Index(['10_EN_3', '28_EN_4', '16_EN_3', '2_EN_3', '31_EN_3', '33_EN_3',
       '8_EN_4', '10_EN_4', '9_EN_4', '3_EN_3', '25_EN_4', '18_EN_4',
       '48_EN_4', '8_EN_3', '10_EN_1', '23_EN_3', '11_EN_4', '3_EN_4',
       '2_EN_4', '9_EN_3'],
      dtype='object', name='chave_loja_buffer_onda')

In [33]:
# Selecionar as n lojas maiores de um único buffer
n = 3
lojas_selecionadas_maiores = list((df_process_capacitado.query('Tipo_de_buffer == "EN"').groupby('chave_loja_buffer_onda')['Peças'].sum().sort_values(ascending=False).head(n)).index)
lojas_selecionadas_maiores

['10_EN_3', '28_EN_4', '16_EN_3']

In [34]:
df_process_capacitado.query(f"chave_loja_buffer_onda in ({lojas_selecionadas_maiores})")

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max
33800,33800,40,2023-10-30,847254655,443,233,32,0.303,10,CARTON_M,...,40_28.0,28_EN_4,3303008,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512
33801,33801,915,2023-10-30,847254621,445,242,33,0.323,10,CARTON_M,...,40_28.0,28_EN_4,3553770,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512
33802,33802,4051,2023-10-30,847254663,431,225,34,0.295,6,CARTON_M,...,40_28.0,28_EN_4,3297150,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512
33803,33803,8438,2023-10-30,847254639,432,248,32,0.310,12,CARTON_M,...,40_28.0,28_EN_4,3428352,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512
33804,33804,8788,2023-10-30,847254680,445,244,33,0.328,5,CARTON_M,...,40_28.0,28_EN_4,3583140,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61584,61584,8067,2023-11-22,878451301,431,234,19,0.215,24,CARTON_M,...,930_283.0,28_EN_4,1916226,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61585,61585,9984,2023-11-22,878451328,424,245,18,0.187,15,CARTON_M,...,930_283.0,28_EN_4,1869840,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61586,61586,1157,2023-11-22,873049031,450,306,23,0.287,2,CARTON_M,...,943_18.0,28_EN_4,3167100,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61587,61587,8666,2023-11-22,873049102,450,315,20,0.307,2,CARTON_M,...,943_18.0,28_EN_4,2835000,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512


In [35]:
df_process_capacitado.query(f"chave_loja_buffer_onda in ({lojas_selecionadas_maiores})")['Peças'].sum()

8739

# 2- Modelagem

## 2.1- CSP - FIFO

Vamos adicionar uma condição em que o tipo de buffer varia conforme a chave da loja

<p>O peso máximo é mesmo, indenpendente o tipo de buffer. Além disso, nesse momento iremos utilizar o volume padrão para cada buffer</p>

Modelagem matemática 

Dado que temos $n$ tipos de itens $i$ para empacotar, para $k$ lojas, cada um com peso $w_{i k}$, um volume $v_{i k}$, um múltiplo $inner_{i k}$ da quantidade de itens a serem selecionados uma demanda $d_{i k}$, para todo $i \in \{1,...,n\}$ e $k \in \{1,...,r\}$, e um número $m$ de mochilas (caixas) diferentes com capacidade volumétrica $cv_{k}$, uma capacidade de peso $cp$ e uma capacidade em número de peças $np_{k}$, além disso, cada item $i$ da loja $k$ tem um tempo em espera $T_{i k}$ no backlog de produção e os itens com tempo de espera maior devem ser alocados na frente, então temos:

$$
\begin{aligned}
\text{minimizar} \quad &\sum_{j = 1}^{m} \sum_{k = 1}^{r} y_{j k} \\
\text{sujeito a} \quad &\sum_{i = 1}^{n} w_{i k} \xi_{i j k}*inner_{i k} \leq cp*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\}\\
&\sum_{i = 1}^{n} v_{i k} \xi_{i j k} *inner_{i k} \leq cv_{k}*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{i = 1}^{n} \xi_{i j k} *inner_{i k} \leq np_{k}*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{j = 1}^{m} \xi_{i j k} *inner_{i k} = d_{i k} \quad \forall i \in \{1,...,n\}, k \in \{1,...,r\}\\
& T_{i k} > T_{i' k} \implies \sum_{j=1}^{m} \xi_{ijk} \geq \sum_{j=1}^{m} \xi_{i'jk} \quad \forall i, i' \in \{1, ..., n\} , k \in \{1,...,r\}\text{ tal que } T_{i k} > T_{i' k} \\
&\xi_{i j k} \geq0 \in \mathbb{Z} \quad \forall i \in \{1,...,n\}, j \in \{1,...,m\} , k \in \{1,...,r\} \\
&y_{j k} \in \{0,1\} \quad \forall j \in \{1,...,m\}, k \in \{1,...,r\}
\end{aligned}
$$


 Seja $\quad$ $\xi_{i j k}$= número de itens tipo $i$ adicionados na caixa $j$, para a loja $k$.


In [67]:
%%ampl_eval
reset;

param n;  # número de tipos de itens
param cp; # capacidade de peso de cada caixa
param r;  # número de lojas (rotas)

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered; # tipos de itens
set K = 1..r ordered; # lojas (rotas)
param w {I,K} >= 0; # peso do item tipo i para a loja k
param v {I,K} >= 0; # volume do item tipo i para a loja k
param inner{I,K} >= 0; # múltiplo da quantidade de itens a serem selecionados
param d {I,K} >= 0; # demanda de itens tipo i para a loja k

param cv {K} >= 0; # capacidade volumétrica de cada loja (rota)
param np {K} >= 0; # capacidade em número de peças para cada loja (rota)
param T {I,K} >= 0; # tempo de espera no backlog de produção

param maxVal := max {i in I, k in K} w[i,k];

param maxVal_peso := max {i in I, k in K} w[i,k];
param pecas_caixa:= floor(cp / maxVal_peso);

param soma := sum{i in I, k in K} d[i, k];
param maxbins := ceil(soma / pecas_caixa);

set J = 1..maxbins ordered;

var e {I,J,K} integer >= 0; # número de itens tipo i na caixa j para a chave loja k
var y {J,K} binary; # 1 se a caixa j é usada para a loja k

minimize FO:  sum {j in J, k in K} y[j,k];

s.t. R1_capacidade_mochila_peso {j in J, k in K}:
   sum {i in I} w[i,k] * e[i,j,k] * inner[i,k] <= cp * y[j,k] suffix block j;

s.t. R2_capacidade_mochila_volume {j in J, k in K}:
   sum {i in I} v[i,k] * e[i,j,k] * inner[i,k] <= cv[k] * y[j,k] suffix block j;
   
s.t. R3_capacidade_mochila_pecas {j in J, k in K}:
   sum {i in I} e[i,j,k] * inner[i,k] <= np[k] * y[j,k] suffix block j;

s.t. R4_quantidade_demanda {i in I, k in K}:
   sum {j in J} e[i,j,k]* inner[i,k] = d[i,k] suffix master 1;

s.t. R5_FIFO_binary {i in I, ip in I, k in K: T[i,k] > T[ip,k]}:
   sum {j in J} e[i, j, k] >= sum {j in J} e[ip, j, k];

### 2.1.1- Importar dados

In [37]:
df_process_situacao2 = df_process_capacitado.query('chave_loja_buffer_onda in ["16_EN_3","10_CX_1","33_EN_3"]')
#df_process_capacitado.query(f"chave_loja_buffer_onda in ({lojas_selecionadas_maiores})")

df_process_situacao2

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max
14,14,1429,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,777_1.0,10_CX_1,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
15,15,1499,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,777_1.0,10_CX_1,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
16,16,1499,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,777_95.0,10_CX_1,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
17,17,909,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,777_95.0,10_CX_1,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
18,18,1499,2023-10-24,539354347,250,100,55,0.180,1,CARTON_M,...,777_95.0,10_CX_1,1375000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58412,58412,761,2023-11-21,869448768,702,246,23,0.147,12,CARTON_L,...,460_137.0,33_EN_3,3971916,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512
58413,58413,761,2023-11-21,869448821,295,247,19,0.167,2,CARTON_M,...,460_137.0,33_EN_3,1384435,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512
58414,58414,761,2023-11-21,869448805,407,253,16,0.080,4,CARTON_M,...,460_137.0,33_EN_3,1647536,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512
58415,58415,761,2023-11-21,848470202,410,380,35,0.200,1,CARTON_M,...,460_471.0,33_EN_3,5453000,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512


In [38]:
delta_t_maximo = df_process_situacao2['tempo_espera_dias'].max()
df_process_situacao2['prioridade'] = df_process_situacao2['tempo_espera_dias'].apply(calcular_prioridade)
df_process_situacao2

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max
14,14,1429,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,777_1.0,10_CX_1,5470920,29,29,27778950.0,83393894.4,75538008.0,83393894.4,170
15,15,1499,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,777_1.0,10_CX_1,5470920,29,29,27778950.0,83393894.4,75538008.0,83393894.4,170
16,16,1499,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,777_95.0,10_CX_1,1750000,29,29,27778950.0,83393894.4,75538008.0,83393894.4,170
17,17,909,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,777_95.0,10_CX_1,1750000,29,29,27778950.0,83393894.4,75538008.0,83393894.4,170
18,18,1499,2023-10-24,539354347,250,100,55,0.180,1,CARTON_M,...,777_95.0,10_CX_1,1375000,29,29,27778950.0,83393894.4,75538008.0,83393894.4,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58412,58412,761,2023-11-21,869448768,702,246,23,0.147,12,CARTON_L,...,460_137.0,33_EN_3,3971916,1,9,27778950.0,83393894.4,75538008.0,75538008.0,512
58413,58413,761,2023-11-21,869448821,295,247,19,0.167,2,CARTON_M,...,460_137.0,33_EN_3,1384435,1,9,27778950.0,83393894.4,75538008.0,75538008.0,512
58414,58414,761,2023-11-21,869448805,407,253,16,0.080,4,CARTON_M,...,460_137.0,33_EN_3,1647536,1,9,27778950.0,83393894.4,75538008.0,75538008.0,512
58415,58415,761,2023-11-21,848470202,410,380,35,0.200,1,CARTON_M,...,460_471.0,33_EN_3,5453000,1,9,27778950.0,83393894.4,75538008.0,75538008.0,512


In [39]:
df_process_situacao2['Peças_max'].unique()

array([170, 512], dtype=int64)

In [40]:
df_process_situacao2.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade_rota', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit', 'tempo_espera_dias',
       'prioridade', 'volume_util_caixa_Caixa_1', 'volume_util_caixa_Caixa_2',
       'volume_util_caixa_Caixa_3', 'volume_util_padrao', 'Peças_max'],
      dtype='object')

### 2.1.2- Entrada de dados no modelo

In [41]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao2['item_loja'] = df_process_situacao2.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,10_CX_1,16_EN_3,33_EN_3
0,1,0.090,0.220,0.207
1,2,0.090,0.167,0.213
2,3,0.200,0.240,0.156
3,4,0.200,0.213,0.150
4,5,0.180,0.185,0.455
...,...,...,...,...
399,400,0.027,0.000,0.000
400,401,0.147,0.000,0.000
401,402,0.137,0.000,0.000
402,403,0.143,0.000,0.000


In [42]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,10_CX_1,16_EN_3,33_EN_3
0,1,5470920.0,3001320.0,3228148.0
1,2,5470920.0,2741220.0,2456160.0
2,3,1750000.0,2864650.0,1212300.0
3,4,1750000.0,2456160.0,2290431.0
4,5,1375000.0,2637180.0,4548648.0
...,...,...,...,...
399,400,670670.0,0.0,0.0
400,401,3555576.0,0.0,0.0
401,402,3482136.0,0.0,0.0
402,403,3961375.0,0.0,0.0


In [43]:
# Variável avaliada:
variavel_ik = 'tempo_espera_dias'

df_matriz_tempo = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_tempo.columns.name = None

df_matriz_tempo

,item_loja,10_CX_1,16_EN_3,33_EN_3
0,1,29.0,22.0,22.0
1,2,29.0,22.0,22.0
2,3,29.0,22.0,22.0
3,4,29.0,22.0,22.0
4,5,29.0,22.0,15.0
...,...,...,...,...
399,400,6.0,0.0,0.0
400,401,6.0,0.0,0.0
401,402,6.0,0.0,0.0
402,403,6.0,0.0,0.0


In [46]:
# Variável avaliada:
variavel_ik = 'prioridade'

df_matriz_prioridade = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_prioridade.columns.name = None

df_matriz_prioridade

,item_loja,10_CX_1,16_EN_3,33_EN_3
0,1,29.0,19.0,19.0
1,2,29.0,19.0,19.0
2,3,29.0,19.0,19.0
3,4,29.0,19.0,19.0
4,5,29.0,19.0,9.0
...,...,...,...,...
399,400,9.0,0.0,0.0
400,401,9.0,0.0,0.0
401,402,9.0,0.0,0.0
402,403,9.0,0.0,0.0


In [44]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao2['item_loja'] = df_process_situacao2.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,10_CX_1,16_EN_3,33_EN_3
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
399,400,1.0,0.0,0.0
400,401,1.0,0.0,0.0
401,402,1.0,0.0,0.0
402,403,1.0,0.0,0.0


In [45]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao2['item_loja'] = df_process_situacao2.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,10_CX_1,16_EN_3,33_EN_3
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,2.0,4.0,1.0
3,4,2.0,2.0,2.0
4,5,1.0,1.0,1.0
...,...,...,...,...
399,400,4.0,0.0,0.0
400,401,9.0,0.0,0.0
401,402,6.0,0.0,0.0
402,403,9.0,0.0,0.0


Capacidade da caixa como variável

In [47]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao2.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,10_CX_1,170.0
1,16_EN_3,512.0
2,33_EN_3,512.0


In [48]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao2.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,10_CX_1,83393894.4
1,16_EN_3,75538008.0
2,33_EN_3,75538008.0


In [49]:
df_process_situacao2.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade_rota', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit', 'tempo_espera_dias',
       'prioridade', 'volume_util_caixa_Caixa_1', 'volume_util_caixa_Caixa_2',
       'volume_util_caixa_Caixa_3', 'volume_util_padrao', 'Peças_max',
       'item_loja'],
      dtype='object')

Características das peças

In [50]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner
t = df_matriz_tempo
prioridade = df_matriz_prioridade

Características da caixa

In [51]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [52]:
cv['volume_util_padrao'].values

array([83393894.39999999, 75538008.        , 75538008.        ])

In [69]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1  # qtde de lojas


v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')
t_ = t.drop(columns='item_loja')

ampl.param["w"] = {(i + 1, k + 1): w.iloc[i, k] for i in range(len(w)) for k in range(len(w.columns))}
ampl.param["T"] = {(i + 1, k + 1): t_.iloc[i, k] for i in range(len(t_)) for k in range(len(t_.columns))}
ampl.param["v"] = {(i + 1, k + 1): v.iloc[i, k] for i in range(len(v)) for k in range(len(v.columns))}
ampl.param["d"] = {(i + 1, k + 1): d.iloc[i, k] for i in range(len(d)) for k in range(len(d.columns))}
ampl.param["inner"] = {(i + 1, k + 1): inner_.iloc[i, k] for i in range(len(inner_)) for k in range(len(inner_.columns))}

ampl.param["cv"] = cv['volume_util_padrao'].values
ampl.param["np"] = np['Peças_max'].values

In [64]:
ampl.getValue('soma')

5827

In [56]:
ampl.getValue('maxbins')

225

In [57]:
ampl.getValue('pecas_caixa')

26

### 2.1.3- Solvers

#### Solver Highs

In [70]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=2 timelimit=600 debug= 1"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 600
  tech:debug = 1
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [3e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 2e+02]
  RHS    [1e+00, 5e+01]
Presolving model
192162 rows, 233550 cols, 73473750 nonzeros  21s
Presolve: Infeasible

Solving report
  Status            Infeasible
  Primal bound      inf
  Dual bound        -inf
  Gap               inf
  Solution status   -
  Timing            24.32 (total)
                    24.06 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)
HiGHS 1.7.0: infeasible problem
0 simplex iterations
0 branching nodes


In [71]:
%%ampl_eval
display d, w, v, inner,T;

:       d      w        v     inner   T     :=
1   1    1   0.09     5470920    1    29
1   2    1   0.22     3001320    1    22
1   3    1   0.207    3228150    1    22
2   1    1   0.09     5470920    1    29
2   2    1   0.167    2741220    1    22
2   3    1   0.213    2456160    1    22
3   1    2   0.2      1750000    1    29
3   2    4   0.24     2864650    1    22
3   3    1   0.156    1212300    1    22
4   1    2   0.2      1750000    1    29
4   2    2   0.213    2456160    1    22
4   3    2   0.15     2290430    1    22
5   1    1   0.18     1375000    1    29
5   2    1   0.185    2637180    1    22
5   3    1   0.455    4548650    1    15
6   1    2   0.18     1375000    1    29
6   2    1   0.18     2668050    1    22
6   3    1   0.437    5118140    1    15
7   1    2   0.2      1350000    1    29
7   2    4   0.193    2458150    1    22
7   3    1   0.46     6075000    1    15
8   1    6   0.1      1412220    1    19
8   2    1   0.14     1049360    1    22
8   3    1

AMPLException: Error executing "solve" command:
error processing set I:
	no value for n

In [ ]:
e_s2_highs = ampl.getVariable('e').getValues().toPandas()
e_s2_highs = e_s2_highs.reset_index()
e_s2_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s2_highs = e_s2_highs.query('valor_multiplo >0')
e_s2_highs#.head(20)

In [ ]:
df_agrupamento_loja1_highs_s2 = e_s2_highs.query('chave_loja_buffer_onda == 1').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja1_highs_s2 = df_agrupamento_loja1_highs_s2.reset_index()
df_agrupamento_loja1_highs_s2.columns.name = None
df_agrupamento_loja1_highs_s2 # as colunas são as caixas

In [ ]:
df_agrupamento_loja1_highs_s2['Id'] = vetores[f"{w.columns[0]}"]
df_agrupamento_loja1_highs_s2['chave_loja_buffer_onda'] = list(w.columns)[0]

df_agrupamento_loja1_highs_s2

In [ ]:
df_agrupamento_loja2_highs_s2 = e_s2_highs.query('chave_loja_buffer_onda == 2').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja2_highs_s2 = df_agrupamento_loja2_highs_s2.reset_index()
df_agrupamento_loja2_highs_s2.columns.name = None
df_agrupamento_loja2_highs_s2['Id'] = vetores[f"{w.columns[1]}"]
df_agrupamento_loja2_highs_s2['chave_loja_buffer_onda'] = list(w.columns)[1]

df_agrupamento_loja2_highs_s2

In [ ]:
y_s2_highs = ampl.getVariable('y').getValues().toPandas()
y_s2_highs = y_s2_highs.reset_index()
y_s2_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s2_highs = y_s2_highs.query('selecao>0')

df_caixas_por_loja_highs_s2 = y_s2_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s2 = df_caixas_por_loja_highs_s2.reset_index()
df_caixas_por_loja_highs_s2.columns.name = None
df_caixas_por_loja_highs_s2 # as colunas são as lojas

In [ ]:
df_caixas_por_loja_highs_s2.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s2

In [ ]:
print('Solver Highs')
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s2[f"{col}"].sum()}')

#### Solver SCIP

In [ ]:
ampl.option["solver"] = "scip"
ampl.option["scip_options"] = "outlev=1 timelimit=600"
ampl.solve()

In [ ]:
e_s2_scip = ampl.getVariable('e').getValues().toPandas()
e_s2_scip = e_s2_scip.reset_index()
e_s2_scip.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s2_scip = e_s2_scip.query('valor_multiplo >0')
e_s2_scip#.head(20)

In [ ]:
df_agrupamento_loja1_scip_s2 = e_s2_scip.query('chave_loja_buffer_onda == 1').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja1_scip_s2 = df_agrupamento_loja1_scip_s2.reset_index()
df_agrupamento_loja1_scip_s2.columns.name = None
df_agrupamento_loja1_scip_s2['Id'] = vetores[f"{w.columns[0]}"]
df_agrupamento_loja1_scip_s2['chave_loja_buffer_onda'] = list(w.columns)[0]

df_agrupamento_loja1_scip_s2

In [ ]:
df_agrupamento_loja2_scip_s2 = e_s2_scip.query('chave_loja_buffer_onda == 2').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja2_scip_s2 = df_agrupamento_loja2_scip_s2.reset_index()
df_agrupamento_loja2_scip_s2.columns.name = None
df_agrupamento_loja2_scip_s2['Id'] = vetores[f"{w.columns[1]}"]
df_agrupamento_loja2_scip_s2['chave_loja_buffer_onda'] = list(w.columns)[1]

df_agrupamento_loja2_scip_s2

In [ ]:
df_agrupamento_loja3_scip_s2 = e_s2_scip.query('chave_loja_buffer_onda == 3').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja3_scip_s2 = df_agrupamento_loja3_scip_s2.reset_index()
df_agrupamento_loja3_scip_s2.columns.name = None
df_agrupamento_loja3_scip_s2['Id'] = vetores[f"{w.columns[2]}"]
df_agrupamento_loja3_scip_s2['chave_loja_buffer_onda'] = list(w.columns)[2]

df_agrupamento_loja3_scip_s2

In [ ]:
y_s2_scip = ampl.getVariable('y').getValues().toPandas()
y_s2_scip = y_s2_scip.reset_index()
y_s2_scip.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s2_scip = y_s2_scip.query('selecao>0')

df_caixas_por_loja_scip_s2 = y_s2_scip.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_scip_s2 = df_caixas_por_loja_scip_s2.reset_index()
df_caixas_por_loja_scip_s2.columns.name = None
df_caixas_por_loja_scip_s2 # as colunas são as lojas

In [ ]:
df_caixas_por_loja_scip_s2.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_scip_s2

In [ ]:
print('Solver SCIP')
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_scip_s2[f"{col}"].sum()}')

## 2.2- CSP - limitação rotas

Modelagem matemática 

Dado que temos $n$ tipos de itens $i$ para empacotar, para $k$ lojas, cada um com peso $w_{i}$, um volume $v_{i}$, um múltiplo $inner_{i k}$ da quantiade de itens a serem selecioner uma demanda $d_{i}$, para todo $i \in \{1,...,n\}$, e um número $m$ de mochilas (caixas) idênticas com capacidade volumétrica $cv$, uma capacidade de peso $cp$ e uma capacidade em número de peças $np$, então temos:

$$
\begin{aligned}
\text{minimizar} \quad &\sum_{j = 1}^{m} \sum_{k = 1}^{r} y_{j k} \\
\text{sujeito a} \quad &\sum_{i = 1}^{n} w_{i k} \xi_{i j k}*inner_{i k} \leq cp*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\}\\
&\sum_{i = 1}^{n} v_{i k} \xi_{i j k} *inner_{i k} \leq cv_{k}*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{i = 1}^{n} \xi_{i j k} *inner_{i k} \leq np_{k}*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{j = 1}^{m} \xi_{i j k} *inner_{i k} = d_{i k} \quad \forall i \in \{1,...,n\}, k \in \{1,...,r\}\\
& \sum_{j = 1}^{m} cv_{k}*y_{j k} \leq Vr_{k} \quad \forall k \in \{1,...,r\}\\
&\xi_{i j k} \geq0 \in \mathbb{Z} \quad \forall i \in \{1,...,n\}, j \in \{1,...,m\} , k \in \{1,...,r\} \\
&y_{j k} \in \{0,1\} \quad \forall j \in \{1,...,m\}, k \in \{1,...,r\}
\end{aligned}
$$


 Seja $\quad$ $\xi_{i j k}$= número de itens tipo $i$ adicionados na mochila $j$, para a loja $k$.


In [167]:
%%ampl_eval
reset;

param n;  # número de tipos de itens
param cp; # capacidade de peso de cada caixa
param r;  # número de lojas (rotas)

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered; # tipos de itens
set K = 1..r ordered; # lojas (rotas)
param w {I,K} >= 0; # peso do item tipo i para a loja k
param v {I,K} >= 0; # volume do item tipo i para a loja k
param inner{I,K} >= 0; # múltiplo da quantidade de itens a serem selecionados
param d {I,K} >= 0; # demanda de itens tipo i para a loja k

param cv {K} >= 0; # capacidade volumétrica de cada loja (rota)
param np {K} >= 0; # capacidade em número de peças para cada loja (rota)
param Vr {K} >= 0; # volume da rota da chave k

param maxVal := max {i in I, k in K} w[i,k];

param maxVal_peso := max {i in I, k in K} w[i,k];
param pecas_caixa:= floor(cp / maxVal_peso);

param soma := sum{i in I, k in K} d[i, k];
param maxbins := ceil(soma / pecas_caixa);

set J = 1..maxbins ordered;

var e {I,J,K} integer >= 0; # número de itens tipo i na caixa j para a chave loja k
var y {J,K} binary; # 1 se a caixa j é usada para a chave loja k

minimize FO:  sum {j in J, k in K} y[j,k];

s.t. R1_capacidade_mochila_peso {j in J, k in K}:
   sum {i in I} w[i,k] * e[i,j,k] * inner[i,k] <= cp * y[j,k] suffix block j;

s.t. R2_capacidade_mochila_volume {j in J, k in K}:
   sum {i in I} v[i,k] * e[i,j,k] * inner[i,k] <= cv[k] * y[j,k] suffix block j;

s.t. R3_capacidade_mochila_pecas {j in J, k in K}:
   sum {i in I} e[i,j,k] * inner[i,k] <= np[k] * y[j,k] suffix block j;

s.t. R4_quantidade_demanda {i in I, k in K}:
   sum {j in J} e[i,j,k]* inner[i,k] <= d[i,k] suffix master 1;

# Limite inferior para o somatório dos volumes das caixas em uma rota
s.t. R5_demanda_inferior {k in K}:
   sum {i in I, j in J} v[i,k] * e[i,j,k] * inner[i,k] <= Vr[k];

s.t. R6_volume_inferior {k in K}:
    sum {j in J} y[j,k] * cv[k] >= Vr[k];

# Limite superior para o somatório dos volumes das caixas em uma rota
s.t. R6_volume_rota_superior {k in K}:
    sum {j in J} y[j,k] * cv[k] <= Vr[k];

### 2.2.1- Importar dados

In [117]:
df_process_capacitado.groupby(['chave_loja_buffer_onda', 'Rota'])['Peças'].sum().sort_values(ascending=False).head(20)

chave_loja_buffer_onda  Rota
10_EN_3                 8       3459
28_EN_4                 10      3187
16_EN_3                 1       2093
2_EN_3                  11      2086
10_CX_1                 8       1948
31_EN_3                 10      1802
33_EN_3                 1       1786
3_CX_9                  5       1690
8_EN_4                  2       1620
3_CX_1                  5       1555
10_EN_4                 8       1509
9_EN_4                  1       1464
10_CX_4                 8       1441
3_EN_3                  5       1403
25_EN_4                 6       1367
31_CX_1                 10      1366
18_EN_4                 3       1360
48_EN_4                 15      1355
8_EN_3                  2       1348
10_EN_1                 8       1329
Name: Peças, dtype: int64

In [118]:
df_process_situacao3 = df_process_capacitado.query('chave_loja_buffer_onda in ["28_EN_4","10_CX_1","3_CX_9"]')

df_process_situacao3

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max
14,14,1429,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,777_1.0,10_CX_1,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
15,15,1499,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,777_1.0,10_CX_1,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
16,16,1499,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,777_95.0,10_CX_1,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
17,17,909,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,777_95.0,10_CX_1,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
18,18,1499,2023-10-24,539354347,250,100,55,0.180,1,CARTON_M,...,777_95.0,10_CX_1,1375000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61584,61584,8067,2023-11-22,878451301,431,234,19,0.215,24,CARTON_M,...,930_283.0,28_EN_4,1916226,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61585,61585,9984,2023-11-22,878451328,424,245,18,0.187,15,CARTON_M,...,930_283.0,28_EN_4,1869840,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61586,61586,1157,2023-11-22,873049031,450,306,23,0.287,2,CARTON_M,...,943_18.0,28_EN_4,3167100,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61587,61587,8666,2023-11-22,873049102,450,315,20,0.307,2,CARTON_M,...,943_18.0,28_EN_4,2835000,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512


In [119]:
df_process_situacao3['Peças_max'].unique()

array([170, 512], dtype=int64)

### 2.2.2- Entrada de dados no modelo

In [120]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao3['item_loja'] = df_process_situacao3.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao3.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,10_CX_1,28_EN_4,3_CX_9
0,1,0.090,0.303,0.220
1,2,0.090,0.323,0.153
2,3,0.200,0.295,0.230
3,4,0.200,0.310,0.230
4,5,0.180,0.328,0.187
...,...,...,...,...
402,403,0.143,0.215,0.000
403,404,0.160,0.187,0.000
404,405,0.000,0.287,0.000
405,406,0.000,0.307,0.000


In [121]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao3.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,10_CX_1,28_EN_4,3_CX_9
0,1,5470920.0,3303008.0,1556100.0
1,2,5470920.0,3553770.0,1681500.0
2,3,1750000.0,3297150.0,1430100.0
3,4,1750000.0,3428352.0,2275200.0
4,5,1375000.0,3583140.0,2096496.0
...,...,...,...,...
402,403,3961375.0,1916226.0,0.0
403,404,3970560.0,1869840.0,0.0
404,405,0.0,3167100.0,0.0
405,406,0.0,2835000.0,0.0


In [122]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao3['item_loja'] = df_process_situacao3.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao3.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,10_CX_1,28_EN_4,3_CX_9
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
402,403,1.0,1.0,0.0
403,404,1.0,1.0,0.0
404,405,0.0,1.0,0.0
405,406,0.0,1.0,0.0


In [123]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao3['item_loja'] = df_process_situacao3.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao3.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,10_CX_1,28_EN_4,3_CX_9
0,1,1.0,10.0,7.0
1,2,1.0,10.0,6.0
2,3,2.0,6.0,17.0
3,4,2.0,12.0,4.0
4,5,1.0,5.0,12.0
...,...,...,...,...
402,403,9.0,24.0,0.0
403,404,6.0,15.0,0.0
404,405,0.0,2.0,0.0
405,406,0.0,2.0,0.0


Capacidade da caixa como variável

In [124]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao3.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,10_CX_1,170.0
1,28_EN_4,512.0
2,3_CX_9,170.0


In [125]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao3.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,10_CX_1,83393894.4
1,28_EN_4,75538008.0
2,3_CX_9,83393894.4


In [126]:
df_process_situacao3[['Id','Data_da_ordem_de_produção','chave_loja_buffer_onda', 'Rota', 'Capacidade_rota']]

,Id,Data_da_ordem_de_produção,chave_loja_buffer_onda,Rota,Capacidade_rota
14,14,2023-10-24,10_CX_1,8,4.000000e+10
15,15,2023-10-24,10_CX_1,8,4.000000e+10
16,16,2023-10-24,10_CX_1,8,4.000000e+10
17,17,2023-10-24,10_CX_1,8,4.000000e+10
18,18,2023-10-24,10_CX_1,8,4.000000e+10
...,...,...,...,...,...
61584,61584,2023-11-22,28_EN_4,10,2.720000e+10
61585,61585,2023-11-22,28_EN_4,10,2.720000e+10
61586,61586,2023-11-22,28_EN_4,10,2.720000e+10
61587,61587,2023-11-22,28_EN_4,10,2.720000e+10


In [127]:
# Variável avaliada:
variavel_ik = 'Capacidade_rota'

df_matriz_caixa_capacidade_rota = df_process_situacao3.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_rota.columns.name = None

df_matriz_caixa_capacidade_rota

,chave_loja_buffer_onda,Capacidade_rota
0,10_CX_1,4.000000e+10
1,28_EN_4,2.720000e+10
2,3_CX_9,2.720000e+10


Características das peças

In [128]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

In [129]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [168]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')
vol_rota = df_matriz_caixa_capacidade_rota.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}
ampl.param["Vr"] = {(k): vol_rota.iloc[k - 1] for k in range(1, len(vol_rota)+1)}

### 2.2.3- Solver

#### Solver Highs

In [169]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=2 timelimit=300 debug= 1"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 300
  tech:debug = 1
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [3e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 3e+01]
  RHS    [1e+00, 3e+10]
Presolving model
3658 rows, 292545 cols, 1460943 nonzeros  0s
3658 rows, 292545 cols, 1460943 nonzeros  82s
Objective function is integral with scale 1

Solving MIP model with:
   3658 rows
   292545 cols (69201 binary, 223344 integer, 0 implied int., 0 continuous)
   1460943 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0    83.3s

Solving report
  Status            Time limit reached
  Primal bound      

In [170]:
%%ampl_eval
display FO;

FO = 0



In [165]:
%%ampl_eval
display e;

e [*,*,1]
:     1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19 :=
1     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
2     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
3     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
4     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
5     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
6     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
7     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
8     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
9     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
10    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
11    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
12    0   0   0   0   0   0

In [150]:
%%ampl_eval
display sum {j in J} e[1,j,1]* inner[1,1];

sum{j in J} e[1,j,1]*inner[1,1] = 0



## 2.3- Modelo sem limitação de rota e FIFO

Vamos realizar a comparação sem a restrição da rota

Modelagem matemática 

Dado que temos $n$ tipos de itens $i$ para empacotar, para $k$ lojas, cada um com peso $w_{i k}$, um volume $v_{i k}$, um múltiplo $inner_{i k}$ da quantidade de itens a serem selecionados uma demanda $d_{i k}$, para todo $i \in \{1,...,n\}$ e $k \in \{1,...,r\}$, e um número $m$ de mochilas (caixas) diferentes com capacidade volumétrica $cv_{k}$, uma capacidade de peso $cp$ e uma capacidade em número de peças $np_{k}$, além disso, cada item $i$ da loja $k$ tem um tempo em espera $T_{i k}$ no backlog de produção e os itens com tempo de espera maior devem ser alocados na frente, então temos:

$$
\begin{aligned}
\text{minimizar} \quad &\sum_{j = 1}^{m} \sum_{k = 1}^{r} y_{j k} \\
\text{sujeito a} \quad &\sum_{i = 1}^{n} w_{i k} \xi_{i j k}*inner_{i k} \leq cp*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\}\\
&\sum_{i = 1}^{n} v_{i k} \xi_{i j k} *inner_{i k} \leq cv_{k}*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{i = 1}^{n} \xi_{i j k} *inner_{i k} \leq np_{k}*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{j = 1}^{m} \xi_{i j k} *inner_{i k} = d_{i k} \quad \forall i \in \{1,...,n\}, k \in \{1,...,r\}\\
&\xi_{i j k} \geq0 \in \mathbb{Z} \quad \forall i \in \{1,...,n\}, j \in \{1,...,m\} , k \in \{1,...,r\} \\
&y_{j k} \in \{0,1\} \quad \forall j \in \{1,...,m\}, k \in \{1,...,r\}
\end{aligned}
$$


 Seja $\quad$ $\xi_{i j k}$= número de itens tipo $i$ adicionados na caixa $j$, para a loja $k$.


In [173]:
%%ampl_eval
reset;

param n;  # número de tipos de itens
param cp; # capacidade de peso de cada caixa
param r;  # número de lojas (rotas)

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered; # tipos de itens
set K = 1..r ordered; # lojas (rotas)
param w {I,K} >= 0; # peso do item tipo i para a loja k
param v {I,K} >= 0; # volume do item tipo i para a loja k
param inner{I,K} >= 0; # múltiplo da quantidade de itens a serem selecionados
param d {I,K} >= 0; # demanda de itens tipo i para a loja k

param cv {K} >= 0; # capacidade volumétrica de cada loja (rota)
param np {K} >= 0; # capacidade em número de peças para cada loja (rota)
param Vr {K} >= 0; # volume da rota da chave k

param maxVal := max {i in I, k in K} w[i,k];

param maxVal_peso := max {i in I, k in K} w[i,k];
param pecas_caixa:= floor(cp / maxVal_peso);

param soma := sum{i in I, k in K} d[i, k];
param maxbins := ceil(soma / pecas_caixa);

set J = 1..maxbins ordered;

var e {I,J,K} integer >= 0; # número de itens tipo i na caixa j para a chave loja k
var y {J,K} binary; # 1 se a caixa j é usada para a chave loja k

minimize FO:  sum {j in J, k in K} y[j,k];

s.t. R1_capacidade_mochila_peso {j in J, k in K}:
   sum {i in I} w[i,k] * e[i,j,k] * inner[i,k] <= cp * y[j,k] suffix block j;

s.t. R2_capacidade_mochila_volume {j in J, k in K}:
   sum {i in I} v[i,k] * e[i,j,k] * inner[i,k] <= cv[k] * y[j,k] suffix block j;

s.t. R3_capacidade_mochila_pecas {j in J, k in K}:
   sum {i in I} e[i,j,k] * inner[i,k] <= np[k] * y[j,k] suffix block j;

s.t. R4_quantidade_demanda {i in I, k in K}:
   sum {j in J} e[i,j,k]* inner[i,k] = d[i,k] suffix master 1;

In [174]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')
vol_rota = df_matriz_caixa_capacidade_rota.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}

In [175]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=300 "
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 300
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [3e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 3e+01]
  RHS    [1e+00, 9e+01]
Presolving model
3655 rows, 292545 cols, 1169289 nonzeros  0s
3655 rows, 292545 cols, 1169289 nonzeros  6s
Objective function is integral with scale 1

Solving MIP model with:
   3655 rows
   292545 cols (69201 binary, 223344 integer, 0 implied int., 0 continuous)
   1169289 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     7.6s
         0       0         0   0.00%   307.3694027     inf                  inf        0     

In [176]:
%%ampl_eval
display FO;

FO = 312



In [177]:
e_s2_highs = ampl.getVariable('e').getValues().toPandas()
e_s2_highs = e_s2_highs.reset_index()
e_s2_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s2_highs = e_s2_highs.query('valor_multiplo >0')
e_s2_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
511,1,171,2,10.0
579,1,194,1,1.0
647,1,216,3,7.0
1027,2,46,2,6.0
1121,2,77,3,1.0
...,...,...,...,...
360760,405,266,2,2.0
361042,406,63,2,1.0
361132,406,93,2,1.0
362017,407,91,2,3.0


In [191]:
vetores_por_loja = {}

# Iterar sobre cada grupo (grupo de cada loja)
for nome_loja, grupo in df_process_situacao3.groupby('chave_loja_buffer_onda'):
    # Extrair os IDs do grupo como um vetor
    vetor_ids = grupo['Id'].tolist()
    # Adicionar o vetor à lista de vetores, usando o nome da loja como chave
    vetores_por_loja[nome_loja] = vetor_ids
    # Atribuir o vetor a uma variável com o nome da loja
    exec(f'vetor_{nome_loja} = vetor_ids')

In [193]:
# Lista dos caracteres finais dos nomes dos vetores
char_final = [col for col in w.columns[:3]]

# Criar o dicionário mapeando os caracteres finais aos vetores
vetores = {char: globals()[f'vetor_{char}'] for char in char_final}

In [195]:
df_agrupamento_loja1_highs_s2 = e_s2_highs.query('chave_loja_buffer_onda == 1').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja1_highs_s2 = df_agrupamento_loja1_highs_s2.reset_index()
df_agrupamento_loja1_highs_s2.columns.name = None
df_agrupamento_loja1_highs_s2['Id'] = vetores[f"{w.columns[0]}"]
df_agrupamento_loja1_highs_s2['chave_loja_buffer_onda'] = list(w.columns)[0]

df_agrupamento_loja1_highs_s2

,item_loja,2,6,9,17,23,27,46,47,52,...,222,229,235,255,257,286,290,297,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14,10_CX_1
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15,10_CX_1
2,3,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16,10_CX_1
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17,10_CX_1
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,10_CX_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31851,10_CX_1
400,401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31852,10_CX_1
401,402,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31853,10_CX_1
402,403,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,31854,10_CX_1


In [196]:
df_agrupamento_loja2_highs_s2 = e_s2_highs.query('chave_loja_buffer_onda == 2').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja2_highs_s2 = df_agrupamento_loja2_highs_s2.reset_index()
df_agrupamento_loja2_highs_s2.columns.name = None
df_agrupamento_loja2_highs_s2['Id'] = vetores[f"{w.columns[1]}"]
df_agrupamento_loja2_highs_s2['chave_loja_buffer_onda'] = list(w.columns)[1]

df_agrupamento_loja2_highs_s2

,item_loja,1,3,4,5,6,8,9,10,11,...,289,290,291,292,293,294,295,297,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33800,28_EN_4
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33801,28_EN_4
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33802,28_EN_4
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33803,28_EN_4
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33804,28_EN_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,403,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,61584,28_EN_4
403,404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61585,28_EN_4
404,405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61586,28_EN_4
405,406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61587,28_EN_4


In [197]:
df_agrupamento_loja3_highs_s2 = e_s2_highs.query('chave_loja_buffer_onda == 2').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja3_highs_s2 = df_agrupamento_loja3_highs_s2.reset_index()
df_agrupamento_loja3_highs_s2.columns.name = None
df_agrupamento_loja3_highs_s2['Id'] = vetores[f"{w.columns[2]}"]
df_agrupamento_loja3_highs_s2['chave_loja_buffer_onda'] = list(w.columns)[2]

df_agrupamento_loja3_highs_s2

ValueError: Length of values (171) does not match length of index (407)

In [198]:
y_s2_highs = ampl.getVariable('y').getValues().toPandas()
y_s2_highs = y_s2_highs.reset_index()
y_s2_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s2_highs = y_s2_highs.query('selecao>0')

df_caixas_por_loja_highs_s2 = y_s2_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s2 = df_caixas_por_loja_highs_s2.reset_index()
df_caixas_por_loja_highs_s2.columns.name = None
df_caixas_por_loja_highs_s2 # as colunas são as lojas

,Caixa,1,2,3
0,1,0.0,1.0,0.0
1,2,1.0,0.0,1.0
2,3,0.0,1.0,1.0
3,4,0.0,1.0,0.0
4,5,0.0,1.0,0.0
...,...,...,...,...
235,292,0.0,1.0,0.0
236,293,0.0,1.0,0.0
237,294,0.0,1.0,0.0
238,295,0.0,1.0,0.0


In [199]:
df_agrupamento_loja2_highs_s2 = e_s2_highs.query('chave_loja_buffer_onda == 2').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja2_highs_s2 = df_agrupamento_loja2_highs_s2.reset_index()
df_agrupamento_loja2_highs_s2.columns.name = None
df_agrupamento_loja2_highs_s2['Id'] = vetores[f"{w.columns[1]}"]
df_agrupamento_loja2_highs_s2['chave_loja_buffer_onda'] = list(w.columns)[1]

df_agrupamento_loja2_highs_s2

,item_loja,1,3,4,5,6,8,9,10,11,...,289,290,291,292,293,294,295,297,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33800,28_EN_4
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33801,28_EN_4
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33802,28_EN_4
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33803,28_EN_4
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33804,28_EN_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,403,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,61584,28_EN_4
403,404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61585,28_EN_4
404,405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61586,28_EN_4
405,406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61587,28_EN_4


In [200]:
y_s2_highs = ampl.getVariable('y').getValues().toPandas()
y_s2_highs = y_s2_highs.reset_index()
y_s2_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s2_highs = y_s2_highs.query('selecao>0')

df_caixas_por_loja_highs_s2 = y_s2_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s2 = df_caixas_por_loja_highs_s2.reset_index()
df_caixas_por_loja_highs_s2.columns.name = None
df_caixas_por_loja_highs_s2 # as colunas são as lojas

,Caixa,1,2,3
0,1,0.0,1.0,0.0
1,2,1.0,0.0,1.0
2,3,0.0,1.0,1.0
3,4,0.0,1.0,0.0
4,5,0.0,1.0,0.0
...,...,...,...,...
235,292,0.0,1.0,0.0
236,293,0.0,1.0,0.0
237,294,0.0,1.0,0.0
238,295,0.0,1.0,0.0


In [201]:
df_caixas_por_loja_highs_s2.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s2

,caixas,10_CX_1,28_EN_4,3_CX_9
0,1,0.0,1.0,0.0
1,2,1.0,0.0,1.0
2,3,0.0,1.0,1.0
3,4,0.0,1.0,0.0
4,5,0.0,1.0,0.0
...,...,...,...,...
235,292,0.0,1.0,0.0
236,293,0.0,1.0,0.0
237,294,0.0,1.0,0.0
238,295,0.0,1.0,0.0


### Caixas

In [202]:
print('Solver Highs')
total_caixas = 0
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s2[col].sum()}')
    total_caixas += df_caixas_por_loja_highs_s2[col].sum()
print(f'\ntotal de caixas: {total_caixas}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "10_CX_1": 42.0
número de caixas para a chave_loja_buffer_onda "28_EN_4": 220.0
número de caixas para a chave_loja_buffer_onda "3_CX_9": 50.0

total de caixas: 312.0
